In [1]:
from pathlib import Path
path = Path('data/time_stamps')

line_range = (2000, 2100)
pixel_range = (5000, 5100)

In [2]:
# Get shape of the SLCs
shape = []
for file in ['lines', 'pixels']:
    with open(path / 'n{}_crp.txt'.format(file)) as f:
        content = f.readlines()
        shape.append(int(content[0]))
shape = tuple(shape)
print(shape)

(9914, 41174)


In [3]:
dir_slcs = [d / 'slc_srd_nl_veenweiden.raw' for d in path.iterdir() if d.is_dir()]

print(dir_slcs)


[PosixPath('data/time_stamps/20141116/slc_srd_nl_veenweiden.raw'), PosixPath('data/time_stamps/20141128/slc_srd_nl_veenweiden.raw'), PosixPath('data/time_stamps/20141210/slc_srd_nl_veenweiden.raw')]


In [4]:
n_time = len(dir_slcs)
n_points = (line_range[1]-line_range[0]) * (pixel_range[1]-pixel_range[0])

print(n_time)
print(n_points)

3
10000


In [5]:
import numpy as np
import xarray as xr

stack=np.empty((n_points, 0))
for d in dir_slcs:
    slc = np.memmap(d, mode='r', dtype='float32',shape=shape)
    stack = np.append(stack, slc[line_range[0]:line_range[1], pixel_range[0]:pixel_range[1]].reshape(n_points, 1), axis=1)

print(stack)
print(stack.shape)

[[ 35.68050003 -43.69489288 -37.2166748 ]
 [-70.70513916  62.954216   101.8187561 ]
 [  6.88615561  45.29898453  -0.14676106]
 ...
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]]
(10000, 3)


In [6]:
minx = 5 
maxx = 10
miny = 50
maxy = 55
dx = 0.5 #Resolution in x
dy = 0.5 #Resolution in y

nx = (maxx - minx)/dx
ny = (maxy - miny)/dy

xx = (np.random.random(n_points)*(maxx - minx) + minx)
yy = (np.random.random(n_points)*(maxy - miny) + miny)
stack_xx = xx.reshape(n_points)
stack_yy = yy.reshape(n_points)

stack_xx.shape

(10000,)

In [7]:
data_xr = xr.Dataset(data_vars=dict(amp=(["points", "time"], stack), x=(["points"],stack_xx),y=(["points"],stack_yy),),
                     coords=dict(
                        points=np.arange(n_points),
                        time=np.arange(n_time),
                        ))
data_xr

<xarray.Dataset>
Dimensions:  (points: 10000, time: 3)
Coordinates:
  * points   (points) int64 0 1 2 3 4 5 6 ... 9993 9994 9995 9996 9997 9998 9999
  * time     (time) int64 0 1 2
Data variables:
    amp      (points, time) float64 35.68 -43.69 -37.22 -70.71 ... 0.0 0.0 0.0
    x        (points) float64 6.948 5.728 5.192 9.73 ... 5.163 7.046 5.538 5.531
    y        (points) float64 54.63 54.75 54.48 52.53 ... 51.37 50.3 53.64 52.07

In [8]:
import stmat
data_xr = data_xr.stm.add_metadata({"name":"test_stm"})

In [9]:
data_xr

<xarray.Dataset>
Dimensions:  (points: 10000, time: 3)
Coordinates:
  * points   (points) int64 0 1 2 3 4 5 6 ... 9993 9994 9995 9996 9997 9998 9999
  * time     (time) int64 0 1 2
Data variables:
    amp      (points, time) float64 35.68 -43.69 -37.22 -70.71 ... 0.0 0.0 0.0
    x        (points) float64 6.948 5.728 5.192 9.73 ... 5.163 7.046 5.538 5.531
    y        (points) float64 54.63 54.75 54.48 52.53 ... 51.37 50.3 53.64 52.07
Attributes:
    name:     test_stm

In [10]:
data_xr.stm

In [11]:
data_xr['amp_mean'] = data_xr.mean(dim='time')['amp']

In [12]:
data_xr.stm.subset(method='threshold', var='amp_mean', threshold='<0.4')

<xarray.Dataset>
Dimensions:   (points: 7456, time: 3)
Coordinates:
  * points    (points) int64 0 4 5 10 12 13 14 ... 9994 9995 9996 9997 9998 9999
  * time      (time) int64 0 1 2
Data variables:
    amp       (points, time) float64 35.68 -43.69 -37.22 3.831 ... 0.0 0.0 0.0
    x         (points) float64 6.948 5.069 6.764 6.762 ... 7.046 5.538 5.531
    y         (points) float64 54.63 53.26 53.12 52.99 ... 50.3 53.64 52.07
    amp_mean  (points) float64 -15.08 -15.72 -23.41 -56.77 ... 0.0 0.0 0.0 0.0
Attributes:
    name:     test_stm

In [13]:
data_xr.stm.subset(method='density', x='x', y='y', dx=0.1, dy=0.1)

grid_cell done
outshape done 50 50
rasterize done


<xarray.Dataset>
Dimensions:   (points: 2433, time: 3)
Coordinates:
  * points    (points) int64 35 46 80 120 241 272 ... 9995 9996 9997 9998 9999
  * time      (time) int64 0 1 2
Data variables:
    amp       (points, time) float64 9.748 10.88 126.4 168.1 ... 0.0 0.0 0.0 0.0
    x         (points) float64 5.051 9.865 5.351 5.831 ... 7.046 5.538 5.531
    y         (points) float64 50.24 52.69 54.77 50.67 ... 50.3 53.64 52.07
    amp_mean  (points) float64 49.0 111.8 54.96 0.0 101.5 ... 0.0 0.0 0.0 0.0
Attributes:
    name:     test_stm